# 🥔 Phase 3: Data Cleaning & Preprocessing

This notebook validates, cleans, and preprocesses the merged potato disease dataset for model training.

**Features:**
- Validates all images and identifies corrupted/unreadable files
- Removes or isolates invalid images
- Standardizes images to consistent format (size, color, normalization)
- Generates detailed cleaning report
- Maintains class-wise folder structure

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
import os

try:
    drive.mount('/content/drive', force_remount=True)
    print('✅ Google Drive mounted successfully!')
except Exception as e:
    print(f'❌ Error: {e}')
    print('\n🔧 Try: Sign out of all accounts, use regular browser, enable cookies')

## 2. Install Dependencies & Configuration

In [ ]:
# Install required packages (if not already installed)
!pip install -q pillow opencv-python-headless tqdm

import os
import shutil
import json
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm

import cv2
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print("✅ Dependencies loaded!")

In [ ]:
# ===== CONFIGURATION =====

# Source: Merged dataset from Phase 2
SOURCE_DIR = "/content/drive/MyDrive/DrukFarm/data/merged_potato_dataset"

# Output: Cleaned and preprocessed dataset
OUTPUT_DIR = "/content/drive/MyDrive/DrukFarm/data/preprocessed_potato_dataset"

# Directory to store corrupted/invalid images (optional backup)
QUARANTINE_DIR = "/content/drive/MyDrive/DrukFarm/data/quarantine_images"

# Preprocessing parameters
TARGET_SIZE = (224, 224)  # Standard CNN input size (e.g., ResNet, VGG, EfficientNet)
COLOR_MODE = "RGB"        # Ensure consistent color format
IMAGE_FORMAT = "jpg"      # Output format
JPEG_QUALITY = 95         # Quality for saved images

# Valid image extensions
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp', '.tiff', '.tif'}

# Whether to quarantine invalid images or delete them
QUARANTINE_INVALID = True  # Set to False to delete instead

print("✅ Configuration loaded!")
print(f"📁 Source: {SOURCE_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"📐 Target size: {TARGET_SIZE}")
print(f"🎨 Color mode: {COLOR_MODE}")

## 3. Image Validation Functions

In [ ]:
def is_valid_extension(filepath: str) -> bool:
    """Check if file has a valid image extension."""
    return Path(filepath).suffix.lower() in VALID_EXTENSIONS


def validate_image(filepath: str) -> tuple:
    """
    Validate an image file.
    Returns: (is_valid, error_message, image_info)
    """
    try:
        # Check file exists
        if not os.path.exists(filepath):
            return False, "File not found", None
        
        # Check file size (skip empty files)
        if os.path.getsize(filepath) == 0:
            return False, "Empty file (0 bytes)", None
        
        # Try opening with PIL
        with Image.open(filepath) as img:
            # Verify the image can be loaded
            img.verify()
        
        # Re-open to get actual data (verify closes the file)
        with Image.open(filepath) as img:
            # Force load pixel data to catch truncated images
            img.load()
            
            # Get image info
            info = {
                'width': img.width,
                'height': img.height,
                'mode': img.mode,
                'format': img.format
            }
            
            # Check minimum dimensions
            if img.width < 10 or img.height < 10:
                return False, f"Image too small ({img.width}x{img.height})", info
            
            return True, None, info
            
    except Exception as e:
        return False, str(e), None


def validate_with_opencv(filepath: str) -> tuple:
    """
    Secondary validation using OpenCV.
    Returns: (is_valid, error_message)
    """
    try:
        img = cv2.imread(filepath)
        if img is None:
            return False, "OpenCV could not read image"
        if img.size == 0:
            return False, "OpenCV read empty image"
        return True, None
    except Exception as e:
        return False, str(e)


print("✅ Validation functions defined!")

## 4. Preprocessing Functions

In [ ]:
def preprocess_image(filepath: str, output_path: str) -> tuple:
    """
    Preprocess a single image:
    - Resize to target dimensions
    - Convert to RGB
    - Save in consistent format
    
    Returns: (success, error_message)
    """
    try:
        # Load image
        with Image.open(filepath) as img:
            # Convert to RGB (handles RGBA, grayscale, palette images)
            if img.mode != 'RGB':
                # Handle transparency by pasting on white background
                if img.mode == 'RGBA':
                    background = Image.new('RGB', img.size, (255, 255, 255))
                    background.paste(img, mask=img.split()[3])
                    img = background
                else:
                    img = img.convert('RGB')
            
            # Resize using high-quality resampling
            img_resized = img.resize(TARGET_SIZE, Image.Resampling.LANCZOS)
            
            # Save with consistent format and quality
            img_resized.save(output_path, 'JPEG', quality=JPEG_QUALITY)
            
            return True, None
            
    except Exception as e:
        return False, str(e)


def get_image_stats(image_path: str) -> dict:
    """
    Get basic statistics about an image.
    """
    try:
        img = np.array(Image.open(image_path))
        return {
            'mean': img.mean(),
            'std': img.std(),
            'min': img.min(),
            'max': img.max()
        }
    except:
        return None


print("✅ Preprocessing functions defined!")

## 5. Scan & Validate Dataset

In [ ]:
def scan_and_validate_dataset(source_dir: str) -> dict:
    """
    Scan the entire dataset and validate all images.
    Returns detailed report of valid and invalid files.
    """
    report = {
        'scan_date': datetime.now().isoformat(),
        'source_directory': source_dir,
        'classes': {},
        'summary': {
            'total_files': 0,
            'valid_images': 0,
            'invalid_images': 0,
            'non_image_files': 0
        },
        'invalid_files': []
    }
    
    if not os.path.exists(source_dir):
        print(f"❌ Source directory not found: {source_dir}")
        return report
    
    # Get all class directories
    class_dirs = [d for d in os.listdir(source_dir) 
                  if os.path.isdir(os.path.join(source_dir, d))]
    
    print(f"📁 Found {len(class_dirs)} classes to validate\n")
    
    for class_name in sorted(class_dirs):
        class_path = os.path.join(source_dir, class_name)
        
        class_report = {
            'valid': [],
            'invalid': [],
            'non_image': [],
            'stats': {
                'total': 0,
                'valid': 0,
                'invalid': 0,
                'non_image': 0
            }
        }
        
        files = os.listdir(class_path)
        print(f"🔍 Validating {class_name}: {len(files)} files...")
        
        for filename in tqdm(files, desc=f"  {class_name}", leave=False):
            filepath = os.path.join(class_path, filename)
            
            # Skip directories
            if os.path.isdir(filepath):
                continue
            
            class_report['stats']['total'] += 1
            report['summary']['total_files'] += 1
            
            # Check extension
            if not is_valid_extension(filepath):
                class_report['non_image'].append(filename)
                class_report['stats']['non_image'] += 1
                report['summary']['non_image_files'] += 1
                continue
            
            # Validate image
            is_valid, error, info = validate_image(filepath)
            
            if is_valid:
                # Double-check with OpenCV
                cv_valid, cv_error = validate_with_opencv(filepath)
                if cv_valid:
                    class_report['valid'].append(filename)
                    class_report['stats']['valid'] += 1
                    report['summary']['valid_images'] += 1
                else:
                    class_report['invalid'].append({
                        'filename': filename,
                        'error': f"OpenCV: {cv_error}"
                    })
                    class_report['stats']['invalid'] += 1
                    report['summary']['invalid_images'] += 1
                    report['invalid_files'].append({
                        'class': class_name,
                        'filename': filename,
                        'path': filepath,
                        'error': f"OpenCV: {cv_error}"
                    })
            else:
                class_report['invalid'].append({
                    'filename': filename,
                    'error': error
                })
                class_report['stats']['invalid'] += 1
                report['summary']['invalid_images'] += 1
                report['invalid_files'].append({
                    'class': class_name,
                    'filename': filename,
                    'path': filepath,
                    'error': error
                })
        
        report['classes'][class_name] = class_report
        print(f"   ✅ Valid: {class_report['stats']['valid']} | ❌ Invalid: {class_report['stats']['invalid']} | ⏭️ Non-image: {class_report['stats']['non_image']}")
    
    return report


# Run validation
print("🔍 DATASET VALIDATION")
print("=" * 50)
validation_report = scan_and_validate_dataset(SOURCE_DIR)

print("\n" + "=" * 50)
print("📊 VALIDATION SUMMARY")
print(f"   Total files scanned: {validation_report['summary']['total_files']}")
print(f"   ✅ Valid images: {validation_report['summary']['valid_images']}")
print(f"   ❌ Invalid images: {validation_report['summary']['invalid_images']}")
print(f"   ⏭️ Non-image files: {validation_report['summary']['non_image_files']}")

## 6. Handle Invalid Images

In [ ]:
def handle_invalid_images(validation_report: dict, quarantine: bool = True) -> dict:
    """
    Handle invalid images by quarantining or deleting them.
    """
    results = {
        'quarantined': [],
        'deleted': [],
        'errors': []
    }
    
    invalid_files = validation_report.get('invalid_files', [])
    
    if not invalid_files:
        print("✅ No invalid images to handle!")
        return results
    
    print(f"\n🔧 Processing {len(invalid_files)} invalid files...")
    
    if quarantine:
        # Create quarantine directory
        os.makedirs(QUARANTINE_DIR, exist_ok=True)
    
    for invalid in tqdm(invalid_files, desc="Processing invalid files"):
        filepath = invalid['path']
        class_name = invalid['class']
        filename = invalid['filename']
        
        try:
            if quarantine:
                # Create class subdirectory in quarantine
                quarantine_class_dir = os.path.join(QUARANTINE_DIR, class_name)
                os.makedirs(quarantine_class_dir, exist_ok=True)
                
                # Move file to quarantine
                dest_path = os.path.join(quarantine_class_dir, filename)
                shutil.move(filepath, dest_path)
                results['quarantined'].append({
                    'original': filepath,
                    'quarantine': dest_path
                })
            else:
                # Delete the file
                os.remove(filepath)
                results['deleted'].append(filepath)
                
        except Exception as e:
            results['errors'].append({
                'file': filepath,
                'error': str(e)
            })
    
    print(f"\n📊 Results:")
    if quarantine:
        print(f"   📦 Quarantined: {len(results['quarantined'])} files")
        print(f"   📁 Quarantine location: {QUARANTINE_DIR}")
    else:
        print(f"   🗑️ Deleted: {len(results['deleted'])} files")
    
    if results['errors']:
        print(f"   ❌ Errors: {len(results['errors'])}")
    
    return results


# Handle invalid images
if validation_report['summary']['invalid_images'] > 0:
    print("\n🔧 HANDLING INVALID IMAGES")
    print("=" * 50)
    cleanup_results = handle_invalid_images(validation_report, quarantine=QUARANTINE_INVALID)
else:
    print("\n✅ No invalid images to handle!")
    cleanup_results = {'quarantined': [], 'deleted': [], 'errors': []}

## 7. Preprocess Valid Images

In [ ]:
def preprocess_dataset(source_dir: str, output_dir: str, validation_report: dict) -> dict:
    """
    Preprocess all valid images in the dataset.
    """
    stats = {
        'processed': 0,
        'failed': 0,
        'classes': {},
        'errors': []
    }
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"📂 Output directory: {output_dir}\n")
    
    for class_name, class_data in validation_report['classes'].items():
        valid_files = class_data['valid']
        
        if not valid_files:
            print(f"⚠️ {class_name}: No valid images to process")
            continue
        
        # Create class output directory
        class_output_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)
        
        class_processed = 0
        class_failed = 0
        
        print(f"🔄 Processing {class_name}: {len(valid_files)} images...")
        
        for filename in tqdm(valid_files, desc=f"  {class_name}", leave=False):
            source_path = os.path.join(source_dir, class_name, filename)
            
            # Generate new filename with consistent extension
            new_filename = Path(filename).stem + f".{IMAGE_FORMAT}"
            output_path = os.path.join(class_output_dir, new_filename)
            
            # Handle potential filename conflicts
            counter = 1
            while os.path.exists(output_path):
                new_filename = f"{Path(filename).stem}_{counter}.{IMAGE_FORMAT}"
                output_path = os.path.join(class_output_dir, new_filename)
                counter += 1
            
            # Preprocess image
            success, error = preprocess_image(source_path, output_path)
            
            if success:
                class_processed += 1
                stats['processed'] += 1
            else:
                class_failed += 1
                stats['failed'] += 1
                stats['errors'].append({
                    'class': class_name,
                    'file': filename,
                    'error': error
                })
        
        stats['classes'][class_name] = {
            'processed': class_processed,
            'failed': class_failed
        }
        print(f"   ✅ Processed: {class_processed} | ❌ Failed: {class_failed}")
    
    return stats


# Preprocess dataset
print("\n🔄 PREPROCESSING DATASET")
print("=" * 50)
print(f"📐 Target size: {TARGET_SIZE}")
print(f"🎨 Color mode: {COLOR_MODE}")
print(f"📄 Output format: {IMAGE_FORMAT}")
print()

preprocessing_stats = preprocess_dataset(SOURCE_DIR, OUTPUT_DIR, validation_report)

## 8. Generate Final Report

In [ ]:
def generate_final_report(validation_report: dict, preprocessing_stats: dict, 
                          cleanup_results: dict, output_dir: str) -> dict:
    """
    Generate comprehensive final report.
    """
    report = {
        'phase': 'Phase 3: Data Cleaning & Preprocessing',
        'timestamp': datetime.now().isoformat(),
        'configuration': {
            'target_size': TARGET_SIZE,
            'color_mode': COLOR_MODE,
            'image_format': IMAGE_FORMAT,
            'source_dir': SOURCE_DIR,
            'output_dir': OUTPUT_DIR
        },
        'validation': {
            'total_files_scanned': validation_report['summary']['total_files'],
            'valid_images': validation_report['summary']['valid_images'],
            'invalid_images': validation_report['summary']['invalid_images'],
            'non_image_files': validation_report['summary']['non_image_files']
        },
        'cleanup': {
            'quarantined': len(cleanup_results.get('quarantined', [])),
            'deleted': len(cleanup_results.get('deleted', [])),
            'errors': len(cleanup_results.get('errors', []))
        },
        'preprocessing': {
            'total_processed': preprocessing_stats['processed'],
            'total_failed': preprocessing_stats['failed'],
            'class_breakdown': preprocessing_stats['classes']
        },
        'output_dataset': {
            'location': output_dir,
            'classes': {}
        }
    }
    
    # Count final output
    if os.path.exists(output_dir):
        for class_name in os.listdir(output_dir):
            class_path = os.path.join(output_dir, class_name)
            if os.path.isdir(class_path):
                count = len([f for f in os.listdir(class_path) if f.endswith(f'.{IMAGE_FORMAT}')])
                report['output_dataset']['classes'][class_name] = count
    
    report['output_dataset']['total_images'] = sum(report['output_dataset']['classes'].values())
    
    # Save report
    report_path = os.path.join(output_dir, 'preprocessing_report.json')
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    return report


# Generate report
final_report = generate_final_report(
    validation_report, preprocessing_stats, cleanup_results, OUTPUT_DIR
)

print("\n" + "=" * 60)
print("📋 FINAL REPORT: Phase 3 Complete")
print("=" * 60)

print(f"\n📅 Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n📊 VALIDATION:")
print(f"   • Scanned: {final_report['validation']['total_files_scanned']} files")
print(f"   • Valid images: {final_report['validation']['valid_images']}")
print(f"   • Invalid/Corrupted: {final_report['validation']['invalid_images']}")

print(f"\n🔄 PREPROCESSING:")
print(f"   • Successfully processed: {final_report['preprocessing']['total_processed']}")
print(f"   • Failed: {final_report['preprocessing']['total_failed']}")
print(f"   • Output size: {TARGET_SIZE[0]}x{TARGET_SIZE[1]} px")

print(f"\n📂 OUTPUT DATASET:")
print(f"   Location: {OUTPUT_DIR}")
print(f"   Total images: {final_report['output_dataset']['total_images']}")
for class_name, count in sorted(final_report['output_dataset']['classes'].items()):
    print(f"   └─ {class_name}: {count} images")

print(f"\n💾 Report saved to: {OUTPUT_DIR}/preprocessing_report.json")

## 9. Verify Preprocessed Dataset

In [ ]:
def verify_preprocessed_dataset(output_dir: str) -> None:
    """
    Verify the preprocessed dataset meets all requirements.
    """
    print("\n🔍 VERIFICATION")
    print("=" * 50)
    
    if not os.path.exists(output_dir):
        print("❌ Output directory does not exist!")
        return
    
    issues = []
    sample_stats = []
    
    for class_name in os.listdir(output_dir):
        class_path = os.path.join(output_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        
        images = [f for f in os.listdir(class_path) if f.endswith(f'.{IMAGE_FORMAT}')]
        
        # Check random samples
        samples = images[:min(5, len(images))]
        for img_name in samples:
            img_path = os.path.join(class_path, img_name)
            with Image.open(img_path) as img:
                if img.size != TARGET_SIZE:
                    issues.append(f"{class_name}/{img_name}: Wrong size {img.size}")
                if img.mode != COLOR_MODE:
                    issues.append(f"{class_name}/{img_name}: Wrong mode {img.mode}")
                
                # Collect stats
                arr = np.array(img)
                sample_stats.append({
                    'class': class_name,
                    'mean': arr.mean(),
                    'std': arr.std()
                })
        
        print(f"✅ {class_name}: {len(images)} images verified")
    
    if issues:
        print(f"\n⚠️ Issues found:")
        for issue in issues[:10]:
            print(f"   • {issue}")
    else:
        print(f"\n✅ All images meet specifications!")
        print(f"   • Size: {TARGET_SIZE[0]}x{TARGET_SIZE[1]}")
        print(f"   • Mode: {COLOR_MODE}")
        print(f"   • Format: {IMAGE_FORMAT.upper()}")
    
    # Show aggregate stats
    if sample_stats:
        means = [s['mean'] for s in sample_stats]
        stds = [s['std'] for s in sample_stats]
        print(f"\n📊 Dataset Statistics (sampled):")
        print(f"   • Mean pixel value: {np.mean(means):.2f}")
        print(f"   • Std pixel value: {np.mean(stds):.2f}")


# Verify
verify_preprocessed_dataset(OUTPUT_DIR)

## 10. Preview Sample Images

In [ ]:
import matplotlib.pyplot as plt
import random

def preview_preprocessed_samples(output_dir: str, samples_per_class: int = 4) -> None:
    """
    Display sample preprocessed images from each class.
    """
    classes = sorted([
        d for d in os.listdir(output_dir)
        if os.path.isdir(os.path.join(output_dir, d))
    ])
    
    if not classes:
        print("No classes found!")
        return
    
    fig, axes = plt.subplots(len(classes), samples_per_class, figsize=(12, 3 * len(classes)))
    
    if len(classes) == 1:
        axes = [axes]
    
    for i, class_name in enumerate(classes):
        class_path = os.path.join(output_dir, class_name)
        images = [f for f in os.listdir(class_path) if f.endswith(f'.{IMAGE_FORMAT}')]
        
        sample_images = random.sample(images, min(samples_per_class, len(images)))
        
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path)
            
            ax = axes[i][j] if len(classes) > 1 else axes[j]
            ax.imshow(img)
            ax.set_title(f"{class_name}\n{TARGET_SIZE[0]}x{TARGET_SIZE[1]}", fontsize=9)
            ax.axis('off')
        
        for j in range(len(sample_images), samples_per_class):
            ax = axes[i][j] if len(classes) > 1 else axes[j]
            ax.axis('off')
    
    plt.tight_layout()
    plt.suptitle("Preprocessed Dataset Samples", fontsize=14, y=1.02)
    plt.show()


# Preview
preview_preprocessed_samples(OUTPUT_DIR)

---

## ✅ Phase 3 Complete!

The cleaned and preprocessed dataset is ready at:
```
/content/drive/MyDrive/DrukFarm/data/preprocessed_potato_dataset/
├── Early_Blight/   (224x224 RGB JPG images)
├── Late_Blight/    (224x224 RGB JPG images)
└── Healthy/        (224x224 RGB JPG images)
```

**Dataset Specifications:**
- ✅ Image size: 224×224 pixels
- ✅ Color format: RGB
- ✅ File format: JPEG
- ✅ All corrupted images removed/quarantined
- ✅ Consistent quality across all images

**Next Steps:**
- Phase 4: Data Augmentation
- Phase 5: Model Training